In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import pandas as pd
from src.tables import get_machines
from src.utils import df_na_vals

In [3]:
raw = get_machines()

In [4]:
cpu_name = raw.cpu_name.str.replace(r'^\d\dth Gen ','')
cpu_name = cpu_name.str.replace(r'^AMD','')
cpu_name = cpu_name.str.replace(r'Threadripper','TR')
cpu_name = cpu_name.str.replace(r'Processor$','')
cpu_name = cpu_name.str.replace(r'-Core','C')
cpu_name = cpu_name.str.replace(r' with Radeon Graphics','')
# cpu_name = raw.cpu_name.str.replace('Core™', '')
raw.cpu_name = cpu_name
raw['cores'] = (raw.cpu_cores / raw.num_gpus).fillna(0).astype(int)
raw['ram'] = (raw.cpu_ram / raw.num_gpus / 1024).round().astype(int)
raw['dph'] = raw.dph_base / raw.num_gpus
raw['disk'] = (raw.disk_space / raw.num_gpus).astype(int)
raw['rented_fraq'] = '[' + raw.num_gpus_rented.astype(str) + "/" + raw.num_gpus.astype(str) + ']'
raw['country'] = raw.location.dropna().apply(lambda loc: loc['country'])
# raw['rented'] = raw.num_gpus_rented > 0
raw['rented'] = raw.num_gpus_rented == raw.num_gpus
raw['min_fraq'] = raw.min_chunk/raw.num_gpus

raw.dlperf = raw.dlperf.round(1)
raw.dlperf_per_dphtotal = raw.dlperf_per_dphtotal.round(1)
raw.disk_bw = raw.disk_bw.round()
raw.duration = pd.to_timedelta(raw.duration * 1e9).dt.days
raw.inet_down = raw.inet_down.fillna(0).astype(int)
raw.inet_up = raw.inet_up.fillna(0).astype(int)
raw.inet_up_cost = (raw.inet_up_cost * 1000).fillna(0).astype(int)
raw.inet_down_cost = (raw.inet_down_cost * 1000).fillna(0).astype(int)


# raw.end_date = (pd.to_datetime(raw.end_date * 1e9) - pd.Timestamp.now()).dt.days
raw.end_date = pd.to_datetime(raw.end_date * 1e9)
raw.start_date = pd.to_datetime(raw.start_date * 1e9)
# raw.set_index('machine_id', inplace=True)

C:\Users\ergot\AppData\Local\Temp\ipykernel_26052\182450783.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  cpu_name = raw.cpu_name.str.replace(r'^\d\dth Gen ','')
C:\Users\ergot\AppData\Local\Temp\ipykernel_26052\182450783.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  cpu_name = cpu_name.str.replace(r'^AMD','')
C:\Users\ergot\AppData\Local\Temp\ipykernel_26052\182450783.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cpu_name = cpu_name.str.replace(r'Processor$','')


In [5]:
cols = ['gpu_name',  'rented_fraq', 'dph', 'credit_discount_max', 'dlperf_per_dphtotal', 'cores', 'ram', 'disk', 'disk_bw',
         'min_chunk', 'country', 'storage_cost', 'reliability2', 'duration', 'inet_up', 'inet_down', 'inet_up_cost', 'inet_down_cost', 'cuda_max_good', 'driver_version',  'pci_gen', 'gpu_lanes', 'hosting_type',  'pcie_bw', 'cpu_name', 'mobo_name',  'location']

raw.query('dph >= 0.5')[cols].sort_values(by='dph', ascending=False)

,gpu_name,rented_fraq,dph,credit_discount_max,dlperf_per_dphtotal,cores,ram,disk,disk_bw,min_chunk,...,inet_down_cost,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,location
397,H100 PCIE,[0/2],3.00,0.4,33.6,64,118,1334,4240.0,1,...,0,12.2,535.54.03,4,16,NaN,16.7,EPYC 7742 64C,MZ52,"{'country': 'US', 'location': 'Ogden, Utah', '..."
52,RTX 3090,[0/4],2.00,0.2,11.7,16,31,411,3641.0,1,...,20,12.2,535.54.03,3,16,NaN,12.4,Ryzen TR PRO 3975WX 32Cs,Pro WS WRX80E,"{'country': 'NO', 'location': 'Drammen, Viken'..."
1323,A100 SXM4,[8/8],2.00,0.4,29.9,32,126,283,4622.0,4,...,0,12.2,535.54.03,4,16,NaN,19.2,EPYC 7763 64C,H12DGO,"{'country': 'US', 'lat': 37.751, 'long': -97.8..."
1322,A100 SXM4,[8/8],2.00,0.4,30.0,32,126,0,5123.0,4,...,0,12.0,525.85.12,4,16,NaN,22.5,EPYC 7763 64C,H12DGO,"{'country': 'US', 'lat': 37.751, 'long': -97.8..."
488,A100 SXM4,[4/4],1.79,0.4,7.9,8,98,38,105.0,1,...,0,12.0,525.125.06,4,16,NaN,16.6,EPYC 7742 64C,440BX Desktop Reference Platform,"{'country': 'CZ', 'location': 'Ostrava, Okres ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034,A40,[5/5],0.50,NaN,67.5,3,50,121,1411.0,1,...,0,11.7,515.86.01,3,16,1.0,6.1,EPYC 7252 8C,Default string,"{'country': 'SE', 'location': 'Umeå, Västerbot..."
672,RTX 4090,[1/1],0.50,0.1,157.9,32,125,1503,4399.0,1,...,2,12.2,535.104.05,4,16,NaN,25.5,Ryzen 9 7950X 16C,X670E VALKYRIE,"{'country': 'JP', 'lat': 35.6897, 'long': 139...."
724,A40,[2/5],0.50,0.4,61.6,3,25,43,1033.0,1,...,0,12.0,525.125.06,3,16,1.0,11.9,EPYC 7252 8C,Default string,"{'country': 'SE', 'location': 'Umeå, Västerbot..."
1010,RTX 4090,[0/4],0.50,0.1,117.2,16,63,186,1891.0,2,...,29,12.0,525.116.04,3,16,NaN,12.6,EPYC 7551P 32C,H11SSL,"{'country': 'RO', 'location': 'Ganeasa, Ilfov'..."


In [11]:
raw.query('num_gpus == 14')[cols]

,gpu_name,rented_fraq,dph,credit_discount_max,dlperf_per_dphtotal,cores,ram,disk,disk_bw,min_chunk,...,inet_down_cost,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,location
290,RTX 4090,[0/14],0.40,0.1,177.5,13,72,457,4532.0,1,...,5,12.2,535.86.05,4,8,NaN,12.5,EPYC 7R12 48C,ROME2D32GM,"{'country': 'TW', 'location': 'Zhongli Distric..."
302,RTX 4090,[9/14],0.30,0.1,248.5,13,36,453,5636.0,1,...,0,12.2,535.86.05,4,8,NaN,12.5,EPYC 7R12 48C,ROME2D32GM,"{'country': 'TW', 'location': 'Zhongli Distric..."
323,RTX 4090,[14/14],0.35,0.1,202.9,13,72,458,5711.0,1,...,5,12.2,535.86.05,4,8,NaN,12.4,EPYC 7R12 48C,ROME2D32GM,"{'country': 'TW', 'location': 'Zhongli Distric..."
332,RTX 4090,[14/14],0.35,0.1,202.9,13,72,450,5574.0,1,...,5,12.2,535.86.05,4,8,NaN,12.8,EPYC 7R12 48C,ROME2D32GM,"{'country': 'TW', 'location': 'Zhongli Distric..."
439,RTX 4090,[14/14],0.55,0.2,63.0,18,36,217,5895.0,14,...,2,12.0,525.105.17,4,16,NaN,12.5,EPYC 7742 64C,ROME2D32GM,"{'country': 'TW', 'location': 'Taipei, Taipei ..."
534,RTX 4090,[0/14],0.55,0.4,104.7,18,36,221,1258.0,1,...,0,12.0,525.125.06,4,8,NaN,12.6,EPYC 7H12 64C,ROME2D32GM,"{'country': 'US', 'location': 'Oklahoma City, ..."
792,RTX 4090,[14/14],0.50,0.2,59.5,13,36,375,4762.0,14,...,0,12.0,525.105.17,4,8,NaN,12.5,EPYC 7R12 48C,ROME2D32GM,"{'country': 'TW', 'location': 'Taipei, Taipei ..."
831,RTX 4090,[14/14],0.55,0.2,56.0,18,36,65,4005.0,14,...,2,12.0,525.105.17,4,8,NaN,12.5,EPYC 7742 64C,ROME2D32GM,"{'country': 'TW', 'location': 'Taoyuan Distric..."


In [6]:
raw[cols].query('country == "RU"')

,gpu_name,rented_fraq,dph,credit_discount_max,dlperf_per_dphtotal,cores,ram,disk,disk_bw,min_chunk,...,inet_down_cost,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,location
1,RTX 3070 Ti,[0/1],0.150,0.40,95.2,12,15,172,418.0,1,...,2,12.2,535.86.05,3,16,NaN,12.3,Core™ i5-10400,ROG STRIX B460,"{'country': 'RU', 'location': 'Cheboksary, Chu..."
57,RTX 4090,[1/1],0.350,0.40,227.9,128,63,1606,4409.0,1,...,2,12.2,535.104.05,4,16,NaN,25.0,EPYC 7B13 64C,ROMED8,"{'country': 'RU', 'location': 'Moscow', 'lat':..."
70,RTX 4090,[0/1],0.350,0.40,189.7,128,126,1650,435.0,1,...,2,12.2,535.104.05,4,16,NaN,24.6,EPYC 7B13 64C,ROMED8,"{'country': 'RU', 'location': 'Moscow', 'lat':..."
79,RTX 3090,[0/4],0.095,0.40,166.2,4,16,407,260.0,2,...,2,12.2,535.86.05,3,8,NaN,4.0,Xeon® E5-2640 v2,None,"{'country': 'RU', 'location': 'Moscow', 'lat':..."
80,RTX 3090,[0/4],0.100,0.40,184.4,4,16,405,240.0,1,...,2,12.2,535.86.05,3,8,NaN,5.3,Xeon® E5-2640 v2,None,"{'country': 'RU', 'location': 'Moscow', 'lat':..."
81,RTX 3090,[4/4],0.095,0.40,166.2,4,16,403,248.0,2,...,2,12.2,535.86.05,3,8,NaN,5.2,Xeon® E5-2640 v2,None,"{'country': 'RU', 'location': 'Moscow', 'lat':..."
82,RTX 3090,[4/4],0.100,0.40,184.4,4,16,407,247.0,1,...,2,12.2,535.86.05,3,8,NaN,5.3,Xeon® E5-2640 v2,None,"{'country': 'RU', 'location': 'Moscow', 'lat':..."
90,RTX 3080 Ti,[0/2],0.150,0.05,166.2,18,63,1665,3210.0,1,...,2,12.0,525.125.06,3,16,NaN,11.8,Xeon® E5-2696 v3,X99,"{'country': 'RU', 'location': 'Maslyaninskiy, ..."
123,RTX 3090,[0/1],0.100,0.40,282.5,16,31,127,722.0,1,...,0,12.2,535.86.05,3,16,NaN,11.3,Xeon® E5-2689 0,E5,"{'country': 'RU', 'location': 'Yaroslavl, Yaro..."
163,RTX 3080,[4/6],0.070,0.40,63.7,2,3,63,416.0,1,...,1,12.2,535.86.05,1,1,NaN,0.1,Core™ i5-10400F,H510 Pro BTC,"{'country': 'RU', 'location': 'Moscow', 'lat':..."
